**Note :** This whole model building process is through sklearn pipeline.This is a very good beginning if you would like to know how in real world models are build.Refer sklearn documentation while working with ColumnTransformer and Sklearn Pipeline if you are not familiar with it already.<br>
<br>
To keep this notebook clear I have not included any Exploratory data analysis part or any fancy plot. You can do plotting and mess with the data as per your wish.<br>

The preprocessed data is pickled and povided here.

Imports

In [185]:
import pandas as pd 
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.metrics import accuracy_score

Imports for model

In [121]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
DT = DecisionTreeClassifier()
RF = RandomForestClassifier()
LR = LogisticRegression()

Imports for feature scaling

In [122]:
from sklearn.preprocessing import MinMaxScaler
minmaxscaler = MinMaxScaler()

Imports for feature selection

In [123]:
from sklearn.feature_selection import SelectKBest,chi2

In [124]:
#Display Pipeline
from sklearn import set_config
set_config(display='diagram')

*IMPORT DATA*

In [125]:
train = pickle.load(open("finaltrain.pkl","rb"))
test = pickle.load(open("finaltest.pkl","rb"))
passengerId = pickle.load(open("passengerId.pkl","rb"))

*Handling Outliers*

Age Column

Conclusion after visualisation<br>
Age column - little skewed and normal distributed<br>
Fare column - highly skewed distribution<br>
Both of the column contains outlier


*Outlier handling using IQR technique*

In [126]:
# Finding the IQR
percentile25 = train['Age'].quantile(0.25)
percentile75 = train['Age'].quantile(0.75)
iqr = percentile75 - percentile25
#Limit for age
High_for_age = percentile75 + 1.5 * iqr
Low_for_age = percentile25 - 1.5 * iqr

In [127]:
#For train data
train1 = train.copy()

train1['Age'] = np.where(
    train1['Age'] > High_for_age,
    High_for_age,
    np.where(
        train1['Age'] < Low_for_age,
        Low_for_age,
        train1['Age']
    )
)

In [128]:
#For test data
test1 = test.copy()

test1['Age'] = np.where(
    test1['Age'] > High_for_age,
    High_for_age,
    np.where(
        test1['Age'] < Low_for_age,
        Low_for_age,
        test1['Age']
    )
)

Fare Column

In [129]:
# Finding the IQR
percentile25 = train['Fare'].quantile(0.25)
percentile75 = train['Fare'].quantile(0.75)
iqr = percentile75 - percentile25
#Limit for fare
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

In [130]:
train2 = train1.copy()

train2['Fare'] = np.where(
    train2['Fare'] > upper_limit,
    upper_limit,
    np.where(
        train2['Fare'] < lower_limit,
        lower_limit,
        train2['Fare']
    )
)

In [131]:
test2 = test1.copy()

test2['Fare'] = np.where(
    test2['Fare'] > upper_limit,
    upper_limit,
    np.where(
        test2['Fare'] < lower_limit,
        lower_limit,
        test2['Fare']
    )
)

Performing train test split

In [132]:
X_train,X_test,y_train,y_test = train_test_split(train2.drop(columns=["Survived"]),train2["Survived"],test_size=0.2,random_state=20)

**Creating transformer using ColumnTransformer for differentcoperation**

*ColumnTransformer for IMPUTATION*

In [133]:
transformer1=ColumnTransformer([
                          ("Mean Imputation",SimpleImputer(),[2]),
                          ("Most frequent imputation",SimpleImputer(strategy="most_frequent"),[4])
],remainder="passthrough") 

*ColumnTransformer for FEATURE ENCODING*

In [134]:
#Transformer for encoding
transformer2 = ColumnTransformer([
                                  ("OHE",OneHotEncoder(sparse=False),[1,3,6])                               
],remainder="passthrough")

*Creating an instance FEATURE SCALING*

In [135]:
#Creating instance for different Scaling techniques
minmaxscaler = MinMaxScaler()

MODEL FITTING WITH PIPE CREATION

*PIPELINE FOR DECISION TREE*

In [168]:

pipe1 = Pipeline([
                 ('imputation',transformer1),
                 ("encoding",transformer2),
                 ("StandardSCaler",minmaxscaler),
                 ('Feature selection',SelectKBest(k=3)),
                 ("model fitting",DT)
])
pipe1.fit(X_train,y_train)
y_pred1 = pipe1.predict(X_test)
accuracy_score(y_test,y_pred1)

0.8324022346368715

In [ ]:
#To display pipeline visually
pipe1.fit(X_train,y_train)

*PIPELINE FOR RANDOM FOREST*

In [167]:
#PIPELINE FOR RANDOM FOREST
pipe2 = Pipeline([
                 ('imputation',transformer1),
                 ("encoding",transformer2),
                 ("Scaling",minmaxscaler),
                 ('Feature selection',SelectKBest(chi2,k=3)),
                 ("model fitting",RF)
])
pipe2.fit(X_train,y_train)
y_pred2 = pipe2.predict(X_test)
accuracy_score(y_test,y_pred2)

0.8324022346368715

In [ ]:
#To display pipeline visually
pipe2.fit(X_train,y_train)

*PIPELINE FOR LOGISTIC REGRESSION*

In [182]:
#PIPELINE FOR LOGISTIC REGRESSION
pipe3 = Pipeline([
                 ('imputation',transformer1),
                 ("encoding",transformer2),
                 ("Scaling",minmaxscaler),
              #   ('Feature selection',SelectKBest(chi2,k=10)),
                 ("model fitting",LR)
])
pipe3.fit(X_train,y_train)
y_pred3 = pipe3.predict(X_test)
accuracy_score(y_test,y_pred3)

0.8715083798882681

In [ ]:
#To display pipeline visually
pipe3.fit(X_train,y_train)

**Sumission dataframe creation**

Decision Tree Model

In [178]:
pipe1.fit(train2.drop(columns=["Survived"]),train2["Survived"])
final_predict = pipe1.predict(test2)
subdata = pd.DataFrame()
subdata["PassengerId"] = passengerId
subdata["Survived"] = final_predict
subdata.to_csv("submission_DT.csv",index=False)

Random Forest Model

In [179]:
pipe2.fit(train2.drop(columns=["Survived"]),train2["Survived"])
final_predict = pipe2.predict(test2)
subdata = pd.DataFrame()
subdata["PassengerId"] = passengerId
subdata["Survived"] = final_predict
subdata.to_csv("submission_RF.csv",index=False)

Logistic Regression Model

In [184]:
pipe3.fit(train2.drop(columns=["Survived"]),train2["Survived"])
final_predict = pipe2.predict(test2)
subdata = pd.DataFrame()
subdata["PassengerId"] = passengerId
subdata["Survived"] = final_predict
subdata.to_csv("submission_LR.csv",index=False)